### Generate a report for Hellinger Distance between real and synthetic random queries

In [67]:
from pathlib import Path
import os
import sys 
import pandas as pd
from csv import writer  
import numpy as np

sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from fuzzy_sql.fuzzy_sql import *


In [68]:
#set paths
root_dir=Path('/home/samer/projects/fuzzy_sql')
real_dir=os.path.join(root_dir,'data/tabular/ready/real')
meta_dir=os.path.join(root_dir,'data/tabular/ready/metadata')
syn_dir=os.path.join(root_dir,'data/tabular/ready/synthetic')

In [69]:
#extract real and synthetic data names
real_names=extract_fnames(real_dir)
real_names.sort()
names_dict=find_syn_fnames(syn_dir, real_names)

Extracted the names of 40 real datasets
Extracted the names of all available synthetic datasets corresponding to 40 real datasets


In [70]:
def summarize_queries(names_dict,no_of_queries):
    syn_dict={} #dictionary of Hellinger and Euclidean Stats per each synthetic trial
    syn_dict['real_name']=[]
    syn_dict['syn_name']=[]
    syn_dict['no_queries']=[]
    syn_dict['hlngr_mean']=[]
    syn_dict['hlngr_median']=[]
    syn_dict['hlngr_stddev']=[]
    syn_dict['ecldn_mean']=[]
    syn_dict['ecldn_median']=[]
    syn_dict['ecldn_stddev']=[]

    for real_name in names_dict:
        real_path=os.path.join(real_dir, real_name+'.csv')
        meta_path=os.path.join(meta_dir, real_name+'.json') #If no corresponding metadata is available, skip tuple
        if not os.path.exists(meta_path): # skip if there is no metadata defined for the dataset
                continue
        if len(names_dict[real_name])==0: #skip if  no synthetic data is available 
            continue
        
        for syn_name in names_dict[real_name]:
            syn_dict['real_name'].append(real_name)
            syn_dict['syn_name'].append(syn_name)
            syn_path=os.path.join(syn_dir, syn_name+'.csv')
            scored_queries=fuzz_tabular(no_of_queries,'twin_aggfltr', real_path, meta_path,syn_path,run_folder='../.runs', print=False)
            syn_dict['no_queries'].append(no_of_queries)
            syn_dict['hlngr_mean'].append(np.nanmean(scored_queries['hlngr_dist']))
            syn_dict['hlngr_median'].append(np.nanmedian(scored_queries['hlngr_dist']))
            syn_dict['hlngr_stddev'].append(np.nanstd(scored_queries['hlngr_dist']))
            syn_dict['ecldn_mean'].append(np.nanmean(scored_queries['ecldn_dist']))
            syn_dict['ecldn_median'].append(np.nanmedian(scored_queries['ecldn_dist']))
            syn_dict['ecldn_stddev'].append(np.nanstd(scored_queries['ecldn_dist']))
            
    return pd.DataFrame(syn_dict)

   

In [71]:
#Generate most abstrcated summary per real dataset
#test_dict={k: names_dict[k] for k in list(names_dict)[:2]}
syn_summary=summarize_queries(names_dict,10)
counts=syn_summary.groupby('real_name').count()['hlngr_mean'].values
real_summary=syn_summary.groupby(['real_name']).mean()
real_summary.insert(1,'no_syn',counts)


Table C1 already exists in the database
Table C1_syn_default_19 already exists in the database
Generated Conditioned Aggregate Query 0 
Generated Conditioned Aggregate Query 1 
Generated Conditioned Aggregate Query 2 
Generated Conditioned Aggregate Query 3 
Generated Conditioned Aggregate Query 4 
Generated Conditioned Aggregate Query 5 
Generated Conditioned Aggregate Query 6 
Generated Conditioned Aggregate Query 7 
Generated Conditioned Aggregate Query 8 
Generated Conditioned Aggregate Query 9 
Generated Conditioned Aggregate Query 10 
Generated Conditioned Aggregate Query 11 
Generated Conditioned Aggregate Query 12 
Generated Conditioned Aggregate Query 13 
Generated Conditioned Aggregate Query 14 
Generated Conditioned Aggregate Query 15 
Generated Conditioned Aggregate Query 16 
Generated Conditioned Aggregate Query 17 
Generated Conditioned Aggregate Query 18 
Generated Conditioned Aggregate Query 19 
Generated Conditioned Aggregate Query 20 
Generated Conditioned Aggregate Q

/tmp/ipykernel_4157/1425033787.py:30: RuntimeWarning: Mean of empty slice
  syn_dict['ecldn_mean'].append(np.nanmean(scored_queries['ecldn_dist']))
/home/samer/projects/fuzzy_sql/.env_dev/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/samer/projects/fuzzy_sql/.env_dev/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Table C19 already exists in the database
Table C19_syn_39 already exists in the database
Generated Conditioned Aggregate Query 0 
Generated Conditioned Aggregate Query 1 
Generated Conditioned Aggregate Query 2 
Generated Conditioned Aggregate Query 3 
Generated Conditioned Aggregate Query 4 
Generated Conditioned Aggregate Query 5 
Generated Conditioned Aggregate Query 6 
Generated Conditioned Aggregate Query 7 
Generated Conditioned Aggregate Query 8 
Generated Conditioned Aggregate Query 9 
Generated Conditioned Aggregate Query 10 
Generated Conditioned Aggregate Query 11 
Generated Conditioned Aggregate Query 12 
Generated Conditioned Aggregate Query 13 
Generated Conditioned Aggregate Query 14 
Generated Conditioned Aggregate Query 15 
Generated Conditioned Aggregate Query 16 
Generated Conditioned Aggregate Query 17 
Generated Conditioned Aggregate Query 18 
Generated Conditioned Aggregate Query 19 
Generated Conditioned Aggregate Query 20 
Generated Conditioned Aggregate Query 2

/tmp/ipykernel_4157/1425033787.py:30: RuntimeWarning: Mean of empty slice
  syn_dict['ecldn_mean'].append(np.nanmean(scored_queries['ecldn_dist']))
/home/samer/projects/fuzzy_sql/.env_dev/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/samer/projects/fuzzy_sql/.env_dev/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Table C2 already exists in the database
Table C2_syn_06 already exists in the database
Generated Conditioned Aggregate Query 0 
Generated Conditioned Aggregate Query 1 
Generated Conditioned Aggregate Query 2 
Generated Conditioned Aggregate Query 3 
Generated Conditioned Aggregate Query 4 
Generated Conditioned Aggregate Query 5 
Generated Conditioned Aggregate Query 6 
Generated Conditioned Aggregate Query 7 
Generated Conditioned Aggregate Query 8 
Generated Conditioned Aggregate Query 9 
Generated Conditioned Aggregate Query 10 
Generated Conditioned Aggregate Query 11 
Generated Conditioned Aggregate Query 12 
Generated Conditioned Aggregate Query 13 
Generated Conditioned Aggregate Query 14 
Generated Conditioned Aggregate Query 15 
Generated Conditioned Aggregate Query 16 
Generated Conditioned Aggregate Query 17 
Generated Conditioned Aggregate Query 18 
Generated Conditioned Aggregate Query 19 
Generated Conditioned Aggregate Query 20 
Generated Conditioned Aggregate Query 21 

...

Table C6 already exists in the database
Table C6_syn_default_10 already exists in the database
Generated Conditioned Aggregate Query 0 
Generated Conditioned Aggregate Query 1 
Generated Conditioned Aggregate Query 2 
Generated Conditioned Aggregate Query 3 
Generated Conditioned Aggregate Query 4 
Generated Conditioned Aggregate Query 5 
Generated Conditioned Aggregate Query 6 
Generated Conditioned Aggregate Query 7 
Generated Conditioned Aggregate Query 8 
Generated Conditioned Aggregate Query 9 
Generated Conditioned Aggregate Query 10 
Generated Conditioned Aggregate Query 11 
Generated Conditioned Aggregate Query 12 
Generated Conditioned Aggregate Query 13 
Generated Conditioned Aggregate Query 14 
Generated Conditioned Aggregate Query 15 
Generated Conditioned Aggregate Query 16 
Generated Conditioned Aggregate Query 17 
Generated Conditioned Aggregate Query 18 
Generated Conditioned Aggregate Query 19 
Generated Conditioned Aggregate Query 20 
Generated Conditioned Aggregate Q

/tmp/ipykernel_4157/1425033787.py:30: RuntimeWarning: Mean of empty slice
  syn_dict['ecldn_mean'].append(np.nanmean(scored_queries['ecldn_dist']))
/home/samer/projects/fuzzy_sql/.env_dev/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/samer/projects/fuzzy_sql/.env_dev/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Table C7 already exists in the database
Table C7_syn_default_13 already exists in the database
Generated Conditioned Aggregate Query 0 
Generated Conditioned Aggregate Query 1 
Generated Conditioned Aggregate Query 2 
Generated Conditioned Aggregate Query 3 
Generated Conditioned Aggregate Query 4 
Generated Conditioned Aggregate Query 5 
Generated Conditioned Aggregate Query 6 
Generated Conditioned Aggregate Query 7 
Generated Conditioned Aggregate Query 8 
Generated Conditioned Aggregate Query 9 
Generated Conditioned Aggregate Query 10 
Generated Conditioned Aggregate Query 11 
Generated Conditioned Aggregate Query 12 
Generated Conditioned Aggregate Query 13 
Generated Conditioned Aggregate Query 14 
Generated Conditioned Aggregate Query 15 
Generated Conditioned Aggregate Query 16 
Generated Conditioned Aggregate Query 17 
Generated Conditioned Aggregate Query 18 
Generated Conditioned Aggregate Query 19 
Generated Conditioned Aggregate Query 20 
Generated Conditioned Aggregate Q

: 

: 

In [1]:
# Generate Report in HTML
with open('../reports/SEP202022/hlngr_queries_frame.html','r') as f:
    html_string=f.read()
with open('../reports/SEP202022/hlngr_queries_report.html', 'w') as f:
    f.write(html_string.format(pandas_table=real_summary.to_html(classes='mystyle')))

NameError: name 'real_summary' is not defined

In [ ]:
#Convert to pdf
with open('../reports/SEP202022/hlngr_queries_report.html','r') as f:
    html_string=f.read()
#html_string=html_string.replace('\n',"")

import weasyprint
pdf=weasyprint.HTML('../reports/SEP202022/hlngr_queries_report.html').write_pdf()
with open('../reports/SEP202022/hlngr_queries_report.pdf','w+b') as f:
    f.write(pdf) 